# Hyperliquid Vault PnL Analysis

In this notebook, we demonstrate how to analyse the historical performance of a [Hyperliquid](https://hyperliquid.xyz/) vault by reconstructing its position history and visualising the equity curve (cumulative PnL).

**What we'll cover:**

- Fetching trade fills from Hyperliquid API for a specific vault
- Reconstructing position events (opens, closes, increases, decreases) from fill data
- Creating an analysis DataFrame with exposure and PnL tracking per market
- Visualising the vault's equity curve using Plotly

**About Hyperliquid Vaults:**

Hyperliquid vaults are managed trading accounts where depositors delegate capital to vault managers who execute perpetual futures trading strategies. Unlike ERC-4626 vaults, Hyperliquid vaults operate on Hyperliquid's L1 chain with off-chain order book matching.

**Note:** The Hyperliquid API has pagination limits (max 10,000 fills), so historical analysis is limited to recent trading activity.

For questions or feedback, contact [Trading Strategy community](https://tradingstrategy.ai/community).

## Setup

- Configure notebook display settings
- Set up Plotly for static image output (for documentation rendering)

In [ ]:
import datetime

import pandas as pd
from plotly.offline import init_notebook_mode
import plotly.io as pio
import plotly.graph_objects as go

pd.options.display.float_format = "{:,.2f}".format
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Set up Plotly chart output
image_format = "png"
width = 1200
height = 600

init_notebook_mode()
pio.renderers.default = image_format

current_renderer = pio.renderers[image_format]
current_renderer.width = width
current_renderer.height = height

## Vault Configuration

We'll analyze the **Trading Strategy - IchiV3 LS** vault, which executes long/short perpetual futures strategies.

You can view this vault on the Hyperliquid app: https://app.hyperliquid.xyz/vaults/0x3df9769bbbb335340872f01d8157c779d73c6ed0

In [ ]:
# Vault address to analyze
VAULT_ADDRESS = "0x3df9769bbbb335340872f01d8157c779d73c6ed0"

# Time range for analysis (last 30 days by default)
END_TIME = datetime.datetime.now()
START_TIME = END_TIME - datetime.timedelta(days=30)

# Display configuration as a table
config_df = pd.DataFrame({
    "Setting": ["Vault Address", "Start Date", "End Date"],
    "Value": [VAULT_ADDRESS, START_TIME.strftime('%Y-%m-%d'), END_TIME.strftime('%Y-%m-%d')]
})
display(config_df.set_index("Setting"))

## Fetch Trade Fills

First, we create an HTTP session configured with retry logic for the Hyperliquid API, then fetch all trade fills for the vault within our time range.

The API returns fills in reverse chronological order, but our fetch function automatically sorts them chronologically for position reconstruction.

In [ ]:
from eth_defi.hyperliquid.session import create_hyperliquid_session
from eth_defi.hyperliquid.position import fetch_vault_fills

# Create session with automatic retry logic
session = create_hyperliquid_session()

# Fetch fills for the vault
fills = list(fetch_vault_fills(
    session,
    VAULT_ADDRESS,
    start_time=START_TIME,
    end_time=END_TIME,
))

print(f"Fetched {len(fills)} fills")

# Show sample fills as a table
if fills:
    sample_data = []
    for fill in fills[:5]:  # Show first 5 fills
        sample_data.append({
            "Coin": fill.coin,
            "Side": fill.side,
            "Size": float(fill.size),
            "Price": float(fill.price),
            "Time": fill.timestamp,
        })
    display(pd.DataFrame(sample_data))

## Reconstruct Position History

The raw fills don't directly tell us about position state. We need to process them chronologically to reconstruct position events:

- **Open**: New position from flat
- **Close**: Position closed to flat (realised PnL)
- **Increase**: Position size increased
- **Decrease**: Partial position reduction (realised PnL)

In [ ]:
from eth_defi.hyperliquid.position import reconstruct_position_history, get_position_summary

# Reconstruct position events from fills
events = list(reconstruct_position_history(fills))

print(f"Reconstructed {len(events)} position events")

# Show position summary per market as a table
summary = get_position_summary(events)

summary_data = []
for coin, stats in sorted(summary.items()):
    summary_data.append({
        "Market": coin,
        "Total Trades": stats['total_trades'],
        "Opens": stats['opens'],
        "Closes": stats['closes'],
        "Realised PnL": float(stats['total_realized_pnl']),
        "Total Fees": float(stats['total_fees']),
    })

summary_df = pd.DataFrame(summary_data).set_index("Market")
display(summary_df)

## Create Analysis DataFrame

Now we convert the position events into a pandas DataFrame suitable for analysis. The DataFrame tracks:

- **Exposure**: Notional value (size × price) for each position direction per market
- **PnL**: Cumulative realised profit/loss for each direction per market

Column naming convention:
- `{coin}_long_exposure` / `{coin}_long_pnl`
- `{coin}_short_exposure` / `{coin}_short_pnl`

In [ ]:
from eth_defi.hyperliquid.position_analysis import create_account_dataframe

# Create the analysis DataFrame
df = create_account_dataframe(events)

# Display DataFrame info as a table
info_df = pd.DataFrame({
    "Metric": ["Rows", "Columns", "Start Time", "End Time"],
    "Value": [df.shape[0], df.shape[1], str(df.index.min()), str(df.index.max())]
})
display(info_df.set_index("Metric"))

# Show the last few rows
display(df.tail())

## Calculate Total Account PnL

The total account PnL at any point in time is the sum of all `*_pnl` columns. This represents the cumulative realised profit/loss across all markets and directions.

In [ ]:
# Find all PnL columns
pnl_columns = [col for col in df.columns if col.endswith('_pnl')]

# Calculate total PnL
df['total_pnl'] = df[pnl_columns].sum(axis=1)

# Show PnL statistics as a table
final_pnl = df['total_pnl'].iloc[-1]
max_pnl = df['total_pnl'].max()
min_pnl = df['total_pnl'].min()

stats_df = pd.DataFrame({
    "Metric": ["Final PnL", "Max PnL", "Min PnL", "PnL Columns"],
    "Value": [f"${final_pnl:,.2f}", f"${max_pnl:,.2f}", f"${min_pnl:,.2f}", len(pnl_columns)]
})
display(stats_df.set_index("Metric"))

## Visualise profit and loss

The equity curve shows how the vault's cumulative realised PnL evolves over time. This is the key metric for evaluating trading performance.

We use Plotly for interactive visualisation (rendered as static image in documentation).

In [ ]:
# Create equity curve chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['total_pnl'],
    mode='lines',
    name='Cumulative PnL',
    line=dict(color='#2ecc71', width=2),
    fill='tozeroy',
    fillcolor='rgba(46, 204, 113, 0.2)',
))

# Add zero line
fig.add_hline(y=0, line_dash="dash", line_color="grey", opacity=0.5)

fig.update_layout(
    title=f"Hyperliquid Vault Equity Curve<br><sub>Vault: {VAULT_ADDRESS[:10]}...{VAULT_ADDRESS[-8:]}</sub>",
    xaxis_title="Date",
    yaxis_title="Cumulative Realised PnL (USD)",
    template="plotly_white",
    hovermode="x unified",
    yaxis=dict(tickformat="$,.0f"),
)

fig.show()

## Per-Market PnL Breakdown

Let's visualise the PnL contribution from each market to understand which assets drove performance.

In [ ]:
# Calculate total PnL per market (combining long and short)
markets = set(col.rsplit('_', 2)[0] for col in pnl_columns)
market_pnl = {}

for market in markets:
    long_col = f"{market}_long_pnl"
    short_col = f"{market}_short_pnl"
    total = 0
    if long_col in df.columns:
        total += df[long_col].iloc[-1]
    if short_col in df.columns:
        total += df[short_col].iloc[-1]
    market_pnl[market] = total

# Sort by absolute PnL
sorted_markets = sorted(market_pnl.items(), key=lambda x: abs(x[1]), reverse=True)

# Create bar chart
fig = go.Figure()

colours = ['#2ecc71' if pnl >= 0 else '#e74c3c' for _, pnl in sorted_markets]

fig.add_trace(go.Bar(
    x=[m[0] for m in sorted_markets],
    y=[m[1] for m in sorted_markets],
    marker_color=colours,
    text=[f"${pnl:,.0f}" for _, pnl in sorted_markets],
    textposition='outside',
))

fig.update_layout(
    title="Realised PnL by Market",
    xaxis_title="Market",
    yaxis_title="Realised PnL (USD)",
    template="plotly_white",
    yaxis=dict(tickformat="$,.0f"),
)

fig.show()